## 一、声学模型设计
本实验借鉴了视觉的经典网络结构，声学模型采用VGGNet的多个卷积层。  
本部分代码在 ```acoustic_model.py```

### 1. VGGNet简介

VGGNet是由牛津大学计算机视觉组合和Google DeepMind公司研究员一起研发的深度卷积神经网络。  
它通过反复的堆叠小型卷积核和最大池化层，成功的构建了16~19层深的卷积神经网络。  
VGGNet获得了ILSVRC2014年比赛的亚军和定位项目的冠军。目前为止，VGGNet依然被用来提取图像的特征。


### 2. VGGNet网络结构
VGG16的网络结构如下图所示：  
其中224x224x3的彩色图表示3通道的长和宽都为224的图像数据，也是网络的输入层；  
白色部分为卷积层，红色部分为池化层（使用最大池化），蓝色部分为全连接层，其中卷积层和全连接层的激活函数都使用relu；  
总的来说，VGG16网络为13层卷积层+5层池化层+3层全连接层而组成。  

<div align="center"><img src="./png/vggnet1.png"/></div>

### 3. 声学模型网络结构
借鉴VGGNet，我们设计的网络结构如下图所示：  

### 2. 自定义模型训练损失函数

**思路简介**

第一步：使用函数```get_pooled_output()```取出BERT的输出。（BERT网络详见PPT）  
第二步：分别取出标准问和扩展问的向量表示  
第三步：计算二者的余弦相似度  
第四步：将正例部分的余弦相似度值减去1，负例的余弦相似度的值不变  
第五步：将第四步的结果取立方，再取绝对值，作为损失函数  


经过训练，当模型收敛时，损失函数最大程度趋近于0；其含义分别如下：  
负例的余弦相似度在（-1,1）范围内中趋近0，即表征负例中的句子的语义趋近独立不相似，  
而正例的相似度在(0,2)范围内趋近于0，即表征正例中的两个句子的语义趋近完全相同  

综上，此损失函数对于语义相似的训练是合理且有效的

**代码实现**

```python
# 以下为添加的余弦相似度损失函数

logits = tf.matmul(output_layer, output_weights, transpose_b=True)
logits = tf.nn.bias_add(logits, output_bias)

# 取出标准问和扩展问的向量表示
feature_a = logits[:cfg.train_batch_size,:]
feature_b = logits[cfg.train_batch_size:,:]

# 计算二者的余弦相似度
nu = feature_a * feature_b
nu = tf.reduce_sum(nu,1)
feature_a2 = feature_a * feature_a
feature_a2 = tf.reduce_sum(feature_a2,1)
feature_a2 = tf.sqrt(feature_a2)
feature_b2 = feature_b * feature_b
feature_b2 = tf.reduce_sum(feature_b2,1)
feature_b2 = tf.sqrt(feature_b2)
de = feature_a2 * feature_b2
inner_product = nu / de     # 得到余弦相似度


target = np.zeros((cfg.train_batch_size),dtype=np.float32)
target[:int(cfg.train_batch_size/2)] = 1.0  # train_batch_size中的前一半数据为负例，后一半的数据为正例
diffs = inner_product - target  # 正例的余弦相似度减去1，使得范围从(-1,1)变成(-2, 0)，负例的余弦相似度减去的是0，其范围不变
diffs = tf.abs(diffs**3)

# 当模型收敛时，负例的余弦相似度在（-1,1）范围内中趋近0，即表征负例中的句子含义独立不相似
# 而正例的相似度在(0,2)范围内趋近于0，即表征正例中的两个句子含义完全相同
loss_op = tf.reduce_mean(diffs)  
```